# Sclera segmentation

In [ ]:
# import libraries
import cv2
import numpy as np

from matplotlib import pyplot as plt
from operator import itemgetter
from scipy import ndimage
from PIL import ImageFilter, Image

from skimage import segmentation, color
from skimage import graph
from skimage.morphology import closing, square, dilation
from skimage.morphology import closing, opening, square
from skimage import exposure

from threshold import threshold, refine
import norm_cuts as nc

In [ ]:
image_path='/path/to/image/'   # modify the image_path to your own dir
assert image_path!='' and image_path!='/path/to/image/', 'Please specify the dataset_path!'

image=plt.imread(image_path)

# Resize the image
new_size = (600, 800)
resized_image = cv2.resize(image, new_size)


In [ ]:
fix_range=0
cuts=10
compactness=10
blur_scale=1
n_cuts=10
n_thresh=0.1
imp_cuts=10
imp_thresh=0.1
imp_comp=6, 
imp_fix=30.0
gamma=1
 
retinex_img = exposure.automatedMSRCR(image)

preprocessed = exposure.preprocess(retinex_img)

plt.figure(figsize=(15, 15))

plt.subplot(141)
plt.title("Original")
plt.imshow(image)

plt.subplot(142)
plt.title("Retinex Image")
plt.imshow(retinex_img)

plt.subplot(143)
plt.title("Preprocessed")
plt.imshow(preprocessed)

plt.show()

In [ ]:

original, kmeans, ncut = nc.nCut(preprocessed, cuts=cuts, 
compactness=compactness, n_cuts=n_cuts, thresh=n_thresh)

plt.figure(figsize=(15, 15))

plt.subplot(144)
plt.title("Original")
plt.imshow(image)

plt.subplot(145)
plt.title("kmeans")
plt.imshow(kmeans)

plt.subplot(146)
plt.title("ncut")
plt.imshow(ncut)

plt.show()

In [ ]:
def calcola_area_e_filtra(image, soglia_y=380):

    image_backup = image.copy()

    #convert in gray scale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #convert in black values lower then 200
    image = np.where(image < 200, 0, image)
    
    # converti in white values greater then 199
    image = np.where(image >= 200, 255, image)

    # label joint regions
    labeled_image, num_labels = ndimage.label(image)
    
    # Apply the y threshold filter to discard pixels below the threshold
    filtered_image = np.copy(image)

    filtered_image[filtered_image > soglia_y] = 0

    # Recalculates the size of connected regions after applying the threshold filter
    filtered_sizes = ndimage.sum(filtered_image, labeled_image, range(1, num_labels + 1))
    
    plt.figure(figsize=(15, 15))

    plt.subplot(147)
    plt.title("Image")
    plt.imshow(image)

    plt.subplot(148)
    plt.title("filtered")
    plt.imshow(filtered_image)

    if len(filtered_sizes) > 0:
        largest_area_index = np.argmax(filtered_sizes)
        
        # Create a mask to keep only the largest area
        largest_area_mask = np.zeros_like(image)
        largest_area_mask[labeled_image == largest_area_index + 1] = 255


        image_output = cv2.bitwise_and(image_backup, image_backup, mask=largest_area_mask)

        return image_output
    else:
        return filtered_image


img_threshold = threshold(retinex_img)

test = calcola_area_e_filtra(ncut)

plt.subplot(149)
plt.title("Image")
plt.imshow(img_threshold)

plt.subplot(150)
plt.title("filtered")
plt.imshow(calcola_area_e_filtra)

plt.show()

In [ ]:

mask_ncut = nc.gaussian_mask(test, img_threshold)

sclera_ncut, mask_ncut = nc.jointRegions(image, ncut, mask_ncut, fix_range, 0)

sclera_ncut, ncut, img_threshold = nc.improve_precision_ncut(original=image,
img_threshold=img_threshold,
seg_img=ncut,
mask=mask_ncut,
preprocessed=preprocessed,
ret_img=retinex_img,
res_img=sclera_ncut,
blur_scale=blur_scale,
cuts=imp_cuts, thresh=imp_thresh, comp=imp_comp,
fix=imp_fix, gamma=gamma)

plt.subplot(151)
plt.title("mask_ncut")
plt.imshow(mask_ncut)

plt.subplot(152)
plt.title("sclera_ncut")
plt.imshow(sclera_ncut)

# convert in gray scale
sclera_ncut = cv2.cvtColor(sclera_ncut, cv2.COLOR_BGR2GRAY)

#convert in white value greater then 0
sclera_ncut = np.where(sclera_ncut > 0, 255, sclera_ncut)

plt.subplot(153)
plt.title("sclera_ncut")
plt.imshow(sclera_ncut)

plt.show()